# Artificial Neural Network (ANN)

### Importing Python modules

In [2]:
import numpy as np
import pandas as pd
import mlflow
import mlflow.tensorflow
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Setting Tracking URI in the current folder

In [3]:
mlflow.set_tracking_uri('')
mlflow.tracking.get_tracking_uri()

''

### Creating an experiment and verifying its existence

In [4]:
#mlflow.create_experiment('Results')
mlflow.get_experiment_by_name('Results')

<Experiment: artifact_location='file:///C:/Users/stevan.stanovic/Desktop/MLFlow%20local/Local/mlruns/1', experiment_id='1', lifecycle_stage='active', name='Results', tags={}>

### Importing data
After importing data, the head of dataset looks like:

![Image of head of Churn Modelling](https://raw.githubusercontent.com/StevanStanovic/mlflow/master/1%20-%20MLFlow%20Tracking%20pour%20un%20algorithme%20supervisé/Images/Head_Churn_Modelling.PNG)

In [5]:
def import_data(path):
    dataset = pd.read_csv(path)
    X = dataset.iloc[:, 3:-1].values
    y = dataset.iloc[:, -1].values
    le = LabelEncoder()
    X[:, 2] = le.fit_transform(X[:, 2])
    ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
    X = np.array(ct.fit_transform(X))
    sc = StandardScaler()
    X = sc.fit_transform(X)
    return (X, y)

### Creating ANN model

In [6]:
def build_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(units=6, activation='relu', input_shape=(12,)))
  model.add(tf.keras.layers.Dense(units=6, activation='relu'))
  model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
  model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return(model)

### Starting a MLFlow experiment

In [7]:
def mlflow_run(exp_id=None, run_name=None):
  
    with mlflow.start_run(run_name=run_name, experiment_id=exp_id) as run:
        
        # Recovering run ID
        run_id = run.info.run_uuid
        
        # Importing and splitting data
        (X, y) = import_data('Churn_Modelling.csv')    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
        
        # Creating classifier
        classifier = build_model()
        
        # Autologging model
        mlflow.tensorflow.autolog()
        
        # Fitting the training set and predicting the test set
        classifier.fit(X_train, y_train, batch_size = 32, epochs = 100)
        y_pred = classifier.predict(X_test)
        y_pred = (y_pred > 0.5)
        
        # Storing confusion matrix metrics in variables
        TP, FP, FN, TN = confusion_matrix(y_test, y_pred).ravel()
        sensitivity_recall_TPR = TP/(TP+FN)
        specificity_TNR = TN/(TN+FP)
        precision_PPV = TP/(TP+FP)
        NPV = TN/(TN+FN)
        fallout_FPR = 1 - specificity_TNR
        FDR = 1 - precision_PPV
        accuracy = (TP+TN)/(y_test.shape[0])
        
        # Logging metrics
        mlflow.log_metric("sensitivity_recall_TPR", sensitivity_recall_TPR)
        mlflow.log_metric("specificity_TNR", specificity_TNR)
        mlflow.log_metric("precision_PPV", precision_PPV)
        mlflow.log_metric("NPV", NPV)
        mlflow.log_metric("fallout_FPR", fallout_FPR)
        mlflow.log_metric("FDR", FDR)
        mlflow.log_metric("accuracy", accuracy)
        
        return run_id

### Running the experiment

In [8]:
if __name__=='__main__':
    # Setting experiment ID and run name
    exp_id = ###Set_your_experiment_ID_using_an_integer
    run_name = "ANN"
    run_id = mlflow_run(exp_id, run_name)
    print("Finished experiment !\n exp_id = {} et run_id = {}".format(exp_id, run_id))

C:\Users\stevan.stanovic\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\tensor_util.py:523: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Epoch 1/100


C:\Users\stevan.stanovic\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if not isinstance(values, collections.Sequence):
C:\Users\stevan.stanovic\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\training\tracking\data_structures.py:718: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):


250/250 [==============================] - 0s 2ms/step - loss: 0.6291 - accuracy: 0.6852
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4898 - accuracy: 0.7955
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4519 - accuracy: 0.8002
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4358 - accuracy: 0.8086
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4279 - accuracy: 0.8105
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4219 - accuracy: 0.8150
Epoch 7/100
250/250 [==============================] - 0s 961us/step - loss: 0.4166 - accuracy: 0.8163
Epoch 8/100
250/250 [==============================] - 0s 993us/step - loss: 0.4114 - accuracy: 0.8191
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.8204
Epoch 10/100
250/250 [==============================] - 0s 968us/step - loss: 0.4033 - accuracy: 0.